In [ ]:
!pip install -U sentence-transformers torch
!pip install datasets

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sentence_transformers import InputExample
from sentence_transformers import losses
from torch.utils.data import DataLoader
from datasets import Dataset
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from google.colab import drive
import shutil

drive.mount('/content/gdrive')

source_path = '/content/gdrive/MyDrive/dataset_text_mining/pre_processed.csv'

destination_path = '/content/dataset_yelp.csv'

shutil.copyfile(source_path, destination_path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


'/content/dataset_yelp.csv'

In [ ]:
df = pd.read_csv('/content/dataset_yelp.csv')
df.head()

,Unnamed: 0,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,...,Mexican,Beauty & Spas,Pubs,Cajun/Creole,Event Planning & Services,Venues & Event Spaces,Bakeries,Hotels,Specialty Food,Main_category
0,4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,...,0,0,0,0,0,0,0,0,0,Sandwiches
1,10,rGQRf8UafX7OTlMNN19I8A,1WHRWwQmZOZDAhp2Qyny4g,uMvVYRgGNXf5boolA9HXTw,5.0,2,0,0,My experience with Shalimar was nothing but wo...,2015-06-21 14:48:06,...,0,0,0,0,0,0,0,0,0,Shopping
2,12,XW_LfMv0fV21l9c6xQd_lw,9OAtfnWag-ajVxRbUTGIyg,lj-E32x9_FA7GmUrBGBEWg,4.0,0,0,0,Love going here for happy hour or dinner! Gre...,2014-06-27 22:44:01,...,0,0,0,0,0,0,0,0,0,Bars
3,13,8JFGBuHMoiNDyfcxuWNtrA,smOvOajNG0lS4Pq7d8g4JQ,RZtGWDLCAtuipwaZ-UfjmQ,4.0,0,0,0,Good food--loved the gnocchi with marinara\nth...,2009-10-14 19:57:14,...,0,0,0,0,0,0,0,0,0,Pizza
4,15,OAhBYw8IQ6wlfw1owXWRWw,1C2lxzUo1Hyye4RFIXly3g,BVndHaLihEYbr76Z0CMEGw,5.0,0,0,0,"Great place for breakfast! I had the waffle, w...",2014-10-11 16:22:06,...,0,0,0,0,0,0,0,0,0,Sandwiches


In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
category_mapping = {'Bars':1, 'Nightlife':2, 'Event Planning & Services':3, 'Mexican':4, 'Breakfast & Brunch':5, 'Pizza':6, 'Sandwiches':7, 'Italian':8, 'Seafood':9,
                    'Coffee & Tea':10, 'Ice Cream & Frozen Yogurt':11, 'Beauty & Spas':12, 'Hotels & Travel':13, 'Shopping':14, 'Burgers':15, 'Sushi Bars':16, 'German':17,
                    'Thai':18, 'Cafes':19}

df["category_label"] = df["Main_category"].map(category_mapping)



In [ ]:
text1 = []
text2 = []
for i in range(10000):
   random_rows = random.sample(df['text'].tolist(), 2)
   text1.append(random_rows[0])
   text2.append(random_rows[1])

df_train = pd.DataFrame({'text1': text1, 'text2': text2})

In [ ]:
texts = df_train['text1'].tolist() + df_train['text2'].tolist()

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)
tfidf_text1 = tfidf_matrix[:len(df_train)]
tfidf_text2 = tfidf_matrix[len(df_train):]

df_train['similarity'] = [
    cosine_similarity(tfidf_text1[i], tfidf_text2[i])[0, 0]
    for i in range(len(df_train))
]

In [ ]:
train_examples = [
    InputExample(texts=[row["text1"], row["text2"]], label=row["similarity"])
    for _, row in df_train.iterrows()
]

In [ ]:
train_loss = losses.CosineSimilarityLoss(model)
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=32)

In [ ]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=4,
    warmup_steps=100,
    output_path='/content/fine_tuned_model'
)

Step,Training Loss
500,0.008100
1000,0.002900


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]